In [1]:
import pandas as pd

In [12]:
spf_file_path = r'C:\Users\yuriy\Desktop\db_tables\EQPDB\clean_excel\SPF_2022_09_12.xlsx'
spf_df = pd.read_excel(spf_file_path)

In [162]:
# df = spf_df.dropna(axis='columns', how='all').drop(spf_df.columns[0:7], axis=1)
df = spf_df[['KKS', 'MCS', 'BLOCK_NUMBER', 'NOMEN_NAME_SPR', 'NAME', 'MARKA_PROT', 'TEC', 'TU', 'ITT', 'CLASS_BEZOPAS', 'GROUP_PNAEG', 'KATEG_SEISMO', 'ARMAT_NP068', 'KATEG_OK', 'ARMAT_KLASS_MAT', 'KOLVO', 'ED_ISM', 'ED_MASSA', 'MASSA_ED', 'MASSA_ALL', 'OBRD_KLASS', 'NOMEN_VID', 'KOD_WD', 'ZDANIE_KKS', 'ELEVATION', 'STORAGECOND', 'ATMTYPE', 'CLIMATEISP_LOC', 'KOMPLEKT_RD']].copy()
df['kks_mcs'] = ''
df.loc[df['KKS'].isnull(), 'kks_mcs'] = 'MCS'
df.loc[df['MCS'].isnull(), 'kks_mcs'] = 'KKS'
df.loc[df['KKS'].isnull(), 'KKS'] = df.loc[df['KKS'].isnull(), 'MCS']

df.loc[df['NAME'].isnull(), 'NAME'] = df.loc[df['NAME'].isnull(), 'NOMEN_NAME_SPR']
df.loc[:, 'ARMAT_KLASS_MAT'] = df['ARMAT_KLASS_MAT'].str.split('/').str[0]
df.loc[:, 'NAME'] = df['NAME'].str.split('/').str[0]

def clean_string(s):
    return ' '.join(s.split())

df = df.map(lambda x: clean_string(x) if isinstance(x, str) else x)

df['kks_len'] = df['KKS'].apply(lambda x: len(str(x)))
df = df[df['kks_len'] >= 12]

#qty_unit
qty_re = {
        'шт': 'pcs',
        'pcs':'psc',
        'кг': 'kg',
        'компл': 'set',
        'Комплект': 'set',
        'к-т': 'set',
        'set': 'set',
        'pc': 'pcs',
        'л': 'l',
        'пар': 'pair',
        'ампула': 'ampule',
        'упак': 'pack',
        'рул': 'pack'
}
for old_value, new_value in (qty_re.items()):
    # print(filtered_df.loc[filtered_df['qty_unit'].str.contains(old_value, case=False, na=False), 'qty_unit'])
    df.loc[df['ED_ISM'].str.contains(old_value, case=False, na=False), 'ED_ISM'] = new_value

df.drop(columns='MCS', inplace=True)
df.drop(columns='NOMEN_NAME_SPR', inplace=True)
df.drop(columns='kks_len', inplace=True)

In [163]:
def clean_string(s):
    if isinstance(s, str):
        s = s.lstrip('-')
        s = s.rstrip(';')
        return s
    else:
        return s

def extract_elevation(s):
    parts = s.split(';')
    if len(parts) >= 2:
        if 'На улице' in parts or 'На крыше' in parts:
            return parts[1]  # Return the right side
    return parts[0]  # Return the left side

df['ELEVATION'] = df['ELEVATION'].apply(clean_string).apply(extract_elevation)



In [133]:
df.iloc[41, 21]

'-10UJA36'

In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 243465 entries, 0 to 250556
Data columns (total 28 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   KKS              243465 non-null  object 
 1   BLOCK_NUMBER     192403 non-null  float64
 2   NAME             237821 non-null  object 
 3   MARKA_PROT       243465 non-null  object 
 4   TEC              112523 non-null  object 
 5   TU               243465 non-null  object 
 6   ITT              243465 non-null  object 
 7   CLASS_BEZOPAS    243465 non-null  object 
 8   GROUP_PNAEG      243465 non-null  object 
 9   KATEG_SEISMO     243465 non-null  object 
 10  ARMAT_NP068      88818 non-null   object 
 11  KATEG_OK         243465 non-null  object 
 12  ARMAT_KLASS_MAT  243465 non-null  object 
 13  KOLVO            243465 non-null  float64
 14  ED_ISM           243465 non-null  object 
 15  ED_MASSA         243465 non-null  object 
 16  MASSA_ED         195206 non-null  float64
 

In [143]:
max_lengths = df.map(lambda x: len(str(x))).max()
max_lengths

KKS                  20
BLOCK_NUMBER          3
NAME               1566
MARKA_PROT           78
TEC                3067
TU                   78
ITT                  42
CLASS_BEZOPAS        10
GROUP_PNAEG           1
KATEG_SEISMO          3
ARMAT_NP068           6
KATEG_OK              3
ARMAT_KLASS_MAT      21
KOLVO                18
ED_ISM                4
ED_MASSA              2
MASSA_ED             20
MASSA_ALL            20
OBRD_KLASS           12
NOMEN_VID             5
KOD_WD               34
ZDANIE_KKS            5
ELEVATION            13
STORAGECOND           3
ATMTYPE               3
CLIMATEISP_LOC       13
KOMPLEKT_RD          17
kks_mcs               3
dtype: int64

In [83]:
df.loc[df['BLOCK_NUMBER'].apply(lambda x: len(str(x)) if pd.notna(x) else 0) > 2, ['BLOCK_NUMBER']]

,BLOCK_NUMBER
0,2.0
1,2.0
2,2.0
3,2.0
4,2.0
...,...
250546,1.0
250550,1.0
250551,1.0
250552,1.0


In [89]:
df.loc[df['NAME'].isnull(), ['KKS', 'kks_mcs', 'NAME']]

,KKS,kks_mcs,NAME
36,12JNG30AA601,KKS,NaN
37,12JNG40AA601,KKS,NaN
38,11JNG20AA601,KKS,NaN
39,11JNG10AA601,KKS,NaN
74,32JNG30AA601,KKS,NaN
...,...,...,...
250387,40KPF90AA002,KKS,NaN
250388,40KPF16AA003,KKS,NaN
250389,40KPG19AA112,KKS,NaN
250422,40KPG16AA102,KKS,NaN


In [ ]:
# filtered1 = df.loc[df['NOMEN_NAME_SPR'].isnull() & df['NAME'].isnull(), 'KKS']
# 
# only_spf = filtered1[~filtered1.isin(eqp_df['kks_code'])]
# only_eqp = eqp_df[~eqp_df['kks_code'].isin(filtered1)]
# common_vals = filtered1[filtered1.isin(eqp_df['kks_code'])]
# 
# eqp_df[eqp_df['kks_code'].isin(filtered1)]

In [164]:
df.loc[: ,'ZDANIE_KKS_2'] = ''
df.loc[df['ZDANIE_KKS'].str.contains('10UJA', na=False, case=False), 'ZDANIE_KKS_2'] = '10UJA'
df.loc[df['ZDANIE_KKS'].str.contains('10UKC', na=False, case=False), 'ZDANIE_KKS_2'] = '10UKC'
df = df[df['ZDANIE_KKS_2'].str.contains('10UJA|10UKC', case=False, na=False )]
df = df[~df['ZDANIE_KKS'].str.contains('UKZ|UMY|UJZ')]
df.loc[df['ZDANIE_KKS'].str.contains('UKC', na=False, case=False), 'ZDANIE_KKS']

644       10UKC
805       10UKC
1175      10UKC
3460      10UKC
3479      10UKC
          ...  
250244    10UKC
250245    10UKC
250246    10UKC
250247    10UKC
250267    10UKC
Name: ZDANIE_KKS, Length: 13283, dtype: object

In [165]:
col_name = ['KKS', 'kks_mcs', 'NAME', 'BLOCK_NUMBER', 'OBRD_KLASS', 'NOMEN_VID', 'ZDANIE_KKS', 'ZDANIE_KKS_2', 'KOMPLEKT_RD', 'KOD_WD',  'ELEVATION', 'TEC', 'TU', 'ITT', 'MARKA_PROT', 'CLASS_BEZOPAS', 'GROUP_PNAEG', 'KATEG_SEISMO', 'ARMAT_NP068', 'KATEG_OK',  'STORAGECOND', 'ATMTYPE', 'CLIMATEISP_LOC', 'ARMAT_KLASS_MAT', 'KOLVO', 'ED_ISM', 'ED_MASSA', 'MASSA_ED', 'MASSA_ALL' ]
df = df[col_name]

df.to_excel(r'C:\Users\yuriy\Desktop\db_tables\EQPDB\clean_excel\spf_clean.xlsx', index=False, float_format="%.4f")